In [10]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [74]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [11]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6945, 7)

In [12]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6652, 5)

In [13]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6951, 9)

fin empresas y comienzo de rescate de pacientes


In [14]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1630916, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [15]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1194478, 4)

In [16]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1195706, 12)

In [17]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [18]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353


In [84]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [85]:
#pacientes_empresas22.shape

In [86]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [87]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [88]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [89]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "BCI"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [90]:
Pacientes_Archivo.head(10)

,HOLDING,PÓLIZA,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,apellidos,nombres,RELACIÓN,PLAN
0,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,18206896,18206896,9,18206896-9,AGUILA URIBE RODRIGO JESUS,AGUILA URIBE,RODRIGO JESUS,Titular Masculino,FULL
1,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,10664709,7,10664709-7,AGUILA VILLARROEL IVAN VLADIMIR,AGUILA VILLARROEL,IVAN VLADIMIR,Titular Masculino,FULL
2,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,21164204,1,21164204-1,AGUILA PAREDES FERNANDA ALEJANDRA,AGUILA PAREDES,FERNANDA ALEJANDRA,Hija,FULL
3,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,23314686,2,23314686-2,AGUILA PAREDES VALENTINA NICOL,AGUILA PAREDES,VALENTINA NICOL,Hija,FULL
4,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,27716797,2,27716797-2,ÁGUILA PAREDES MARÍA IGNACIA,ÁGUILA PAREDES,MARÍA IGNACIA,Hija,FULL
5,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,16161956,6,16161956-6,PAREDES CONTRERAS LUISA ALEJANDRA,PAREDES CONTRERAS,LUISA ALEJANDRA,Esposa,FULL
6,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,14457460,14457460,5,14457460-5,AGUILERA RUBIO MARTA ALICIA,AGUILERA RUBIO,MARTA ALICIA,Titular Femenino,FULL
7,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,12344569,12344569,4,12344569-4,ALMONACID MANCILLA ALEJANDRO TOMAS,ALMONACID MANCILLA,ALEJANDRO TOMAS,Titular Masculino,FULL
8,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,12344569,22302391,6,22302391-6,ALMONACID PILLANCARI DAVID,ALMONACID PILLANCARI,DAVID,Hijo,FULL
9,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,12344569,11714628,6,11714628-6,PILLANCARI BARRIA LUZ ELIANA,PILLANCARI BARRIA,LUZ ELIANA,Esposa,FULL


In [91]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [92]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,HOLDING,PÓLIZA,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,apellidos,nombres,RELACIÓN,PLAN,rut
0,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,18206896,18206896,9,18206896-9,AGUILA URIBE RODRIGO JESUS,AGUILA URIBE,RODRIGO JESUS,Titular Masculino,FULL,182068969
1,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,10664709,7,10664709-7,AGUILA VILLARROEL IVAN VLADIMIR,AGUILA VILLARROEL,IVAN VLADIMIR,Titular Masculino,FULL,106647097
2,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,21164204,1,21164204-1,AGUILA PAREDES FERNANDA ALEJANDRA,AGUILA PAREDES,FERNANDA ALEJANDRA,Hija,FULL,211642041
3,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,23314686,2,23314686-2,AGUILA PAREDES VALENTINA NICOL,AGUILA PAREDES,VALENTINA NICOL,Hija,FULL,233146862
4,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,27716797,2,27716797-2,ÁGUILA PAREDES MARÍA IGNACIA,ÁGUILA PAREDES,MARÍA IGNACIA,Hija,FULL,277167972


In [93]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9824\485331713.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,HOLDING,PÓLIZA,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,apellidos,nombres,RELACIÓN,PLAN,rut
0,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,18206896,18206896,9,18206896-9,AGUILA URIBE RODRIGO JESUS,AGUILA URIBE,RODRIGO JESUS,Titular Masculino,FULL,182068969
1,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,10664709,7,10664709-7,AGUILA VILLARROEL IVAN VLADIMIR,AGUILA VILLARROEL,IVAN VLADIMIR,Titular Masculino,FULL,106647097
2,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,21164204,1,21164204-1,AGUILA PAREDES FERNANDA ALEJANDRA,AGUILA PAREDES,FERNANDA ALEJANDRA,Hija,FULL,211642041
3,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,23314686,2,23314686-2,AGUILA PAREDES VALENTINA NICOL,AGUILA PAREDES,VALENTINA NICOL,Hija,FULL,233146862
4,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,27716797,2,27716797-2,ÁGUILA PAREDES MARÍA IGNACIA,ÁGUILA PAREDES,MARÍA IGNACIA,Hija,FULL,277167972


In [94]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9824\2118328728.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9824\2118328728.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [95]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(22565, 24)

In [96]:
Pacientes_con_id.head()

,HOLDING,PÓLIZA,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,apellidos,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,18206896,18206896,9,18206896-9,AGUILA URIBE RODRIGO JESUS,Aguila Uribe,...,True,0018c00002ay1CCAAY,182068969,Rodrigo Jesus Aguila Uribe,RUT,None,None,None,Hombre,None
1,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,10664709,7,10664709-7,AGUILA VILLARROEL IVAN VLADIMIR,Aguila Villarroel,...,True,0018c00002ay1AnAAI,106647097,Ivan Vladimir Aguila Villarroel,RUT,None,None,None,Hombre,None
2,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,21164204,1,21164204-1,AGUILA PAREDES FERNANDA ALEJANDRA,Aguila Paredes,...,True,0018c00002ay1AlAAI,211642041,Fernanda Alejandra Aguila Paredes,RUT,None,None,None,Mujer,None
3,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,23314686,2,23314686-2,AGUILA PAREDES VALENTINA NICOL,Aguila Paredes,...,True,0018c00002ay4cMAAQ,233146862,Valentina Nicol Aguila Paredes,RUT,None,None,None,Mujer,None
4,DANISCO,995904,DANISCO CHILE S.A. .,TRABAJADORES DANISCO CHILE,10664709,27716797,2,27716797-2,ÁGUILA PAREDES MARÍA IGNACIA,Águila Paredes,...,True,0018c00002giuvRAAQ,277167972,María Ignacia Águila Paredes,RUT,None,None,None,None,None


In [98]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"].astype(str).str.upper()
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"].apply(str).str.strip().replace("+","")
    Subir_pacientes['Phone'] = Subir_pacientes['Phone'].str.title().replace("NaN", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtsEAE'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [99]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "SURA": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [100]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
157385,a1Y8c00000C6V01EAF,0018c00002amPZPAA2,None,None,0018c00002axQgwAAE,Trafigura Chile Ltda. Dental,Eliminar Empresa,BCI,Trafigura Chile Ltda. Dental,Eliminar Empresa,EMP6121,NaN
157386,a1Y8c00000C6V01EAF,0018c00002amPZPAA2,None,None,0018c00002axQgyAAE,Impala Term. G. UF 25 Tit. Rech. Cargas Normales,Eliminar Empresa,BCI,Impala Term. G. UF 25 Tit. Rech. Cargas Normales,Eliminar Empresa,EMP6123,NaN
157387,a1Y8c00000C6V01EAF,0018c00002amPZPAA2,None,None,0018c00002axQgzAAE,Steinweg Brokers SPA,Eliminar Empresa,BCI,Steinweg Brokers SPA,Eliminar Empresa,EMP6124,NaN
157388,a1Y8c00000C6V01EAF,0018c00002amPZPAA2,None,None,0018c00002axQh0AAE,Steinweg Brokers SPA,Eliminar Empresa,BCI,Steinweg Brokers SPA,Eliminar Empresa,EMP6125,NaN
157389,a1Y8c00000C6V01EAF,0018c00002amPZPAA2,None,None,0018c00002axQgdAAE,Cemin,Eliminar Empresa,BCI,Cemin,Eliminar Empresa,EMP6250,NaN


In [101]:
campaña_seleccionada.shape

(22443, 12)

In [102]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [103]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [104]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
